In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit,col,explode
import pyspark.sql.functions as f 


In [3]:
# Extract

spark = SparkSession.builder.appName("ETL Pipeline").getOrCreate()


In [4]:
spark

In [5]:
df = spark.read.text('data/WordData1.txt')

In [6]:
df.show()

+--------------------+
|               value|
+--------------------+
|This is a Japanes...|
|The team members ...|
|As the years pass...|
|If you don't like...|
|He was disappoint...|
|When he encounter...|
|Situps are a terr...|
|Toddlers feeding ...|
|Edith could decid...|
|Her daily goal wa...|
|Tomorrow will bri...|
|His son quipped t...|
|He wondered why a...|
|If my calculator ...|
|The hummingbird's...|
|He went on a whis...|
|This is the last ...|
|I come from a tri...|
|The delicious aro...|
|Weather is not tr...|
+--------------------+
only showing top 20 rows



In [7]:
# transformation
df2 = df.withColumn('splitedData',f.split('value',' '))
df3 = df2.withColumn('words',explode('splitedData'))
word_df = df3.select('words')
word_df.show()

+--------+
|   words|
+--------+
|    This|
|      is|
|       a|
|Japanese|
|    doll|
|     The|
|    team|
| members|
|    were|
|    hard|
|      to|
|    tell|
|   apart|
|   since|
|    they|
|     all|
|    wore|
|   their|
|    hair|
|      in|
+--------+
only showing top 20 rows



In [8]:
word_count = word_df.groupBy('words').count()

In [9]:
word_count.show()

+-----------+-----+
|      words|count|
+-----------+-----+
|   Tomorrow|    4|
|         If|    8|
|      leave|    4|
|      corny|    4|
|        day|    4|
|preoccupied|    4|
|       even|    8|
|      crazy|    4|
|    bananas|    4|
|     priest|    4|
|        did|    4|
|    whether|    4|
|     Having|    4|
|        I'm|    4|
|      crime|    4|
|  surprised|    4|
|      James|    4|
|      could|    8|
|        buy|    4|
|        him|    8|
+-----------+-----+
only showing top 20 rows



In [19]:
# LOAD 
driver = "com.mysql.jdbc.Driver"
url = 'jdbc:mysql://database-2.cjbysn8oqrq1.us-east-1.rds.amazonaws.com/'
table = 'avinash_schema.wordcount'
user='admin'
password='ad789min'

word_count.write.mode('append').format("jdbc").option("url",url).option("driver",driver)\
    .option("dbtable",table).option("mode","append").option("user",user)\
        .option("password",password).save()

In [18]:
# LOAD 
driver1 = "com.mysql.jdbc.Driver"
url1 = 'jdbc:mysql://localhost:3306/'
table1 = 'study.wordcount'
user1='root'
password1='root'

word_count.write.mode('append').format('jdbc').option("url",url1)\
    .option("driver",driver1)\
    .option('dbtable',table1).option('mode','append').option('user',user1)\
        .option('password',password1).save()

In [1]:
# from pyspark.sql import SparkSession

# spark = SparkSession\
#     .builder\
#     .appName("Word Count")\
#     .getOrCreate()

dataframe_mysql = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/exam")\
    .option("driver", "com.mysql.cj.jdbc.Driver")\
    .option("dbtable", "books").option("user", "root")\
    .option("password", "root").load()

print(dataframe_mysql.columns)

NameError: name 'spark' is not defined

In [17]:
dataframe_mysql.show()

+-------+--------------------+-----+
|book_id|               title|price|
+-------+--------------------+-----+
|  12345|  Python Programming|   29|
|  12346|         Learn MySQL|   23|
|  12347|Data Science Cook...|   27|
|  12367|           My dreams|   50|
|  13456|       Eat that frog|   34|
+-------+--------------------+-----+

